# Модули


In [1]:
import pandas as pd
import numpy as np 
import os
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: GeForce GT 540M (CNMeM is disabled, cuDNN not available)


Эта функция очищает data frame от спикеров, которые говорили очень мало и мусорных кандидатов



In [2]:
def clean_df(df, x = 1, y = 15): 
    ind = df.Speaker.value_counts()[x:y].index 
    return df[(df.Speaker.isin(ind) ) & (~ df.Speaker.isin(['CANDIDATES','OTHER']))]

# Подготовка данных


Загружаем датасет и очищаем его

In [3]:
data = pd.read_csv('primary_debates_cleaned.csv')
data = clean_df(data)
data.head(5)

,Line,Speaker,Text,Date,Party,Location,URL
6,7,Clinton,Thank you.,2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
10,11,Sanders,"Well, Gwen and Judy, thank you very much for h...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
14,15,Clinton,I'm running for president to knock down all th...,2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
20,21,Sanders,"Well, to put that in a context, Judy, I think ...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
22,23,Sanders,"... Of course there will be a limit, but when ...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...


In [4]:
EMBEDDING_DIM = 100 # длина вектора, которым мы кодируем слово
MAX_SEQ_LEN = 255   # установка формы текста - 255 слов 

In [5]:
LE = LabelEncoder()

Y_l_encoded = LE.fit_transform(data.Speaker.values) # заменяем каждое название класса на численное значение

labels_index = {}

for index,text in enumerate(data.Text.values):
    
    # берем лэйбл для текста, переведенный в число из Y_l_encoded (index текста = index лэйбла)
    label_id = Y_l_encoded[index]

    labels_index[text] = label_id # добавляем в словарик пару, где ключ - текст, значение - номер лэйбла

labels = Y_l_encoded
texts = data.Text.values
print('Found %s texts.' % len(texts))

Found 5634 texts.


In [6]:
tokenizer = Tokenizer(nb_words=10000)  
tokenizer.fit_on_texts(texts)     # учим токенайзер
sequences = tokenizer.texts_to_sequences(texts) # переводим текст в последовательности

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# если слов меньше 255 - дополняем 0-ми, в противном случае - обрезаем
X = pad_sequences(sequences, maxlen=MAX_SEQ_LEN)  

# one hot encoding классов (лэйблов)
labels = to_categorical(np.asarray(labels))

# разбиваем данные на train и test, перемешав
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
# задаем часть тестовой выборки - 0.3
nb_validation_samples = int(0.3 * X.shape[0])

X_train = X[:-nb_validation_samples]
Y_train = labels[:-nb_validation_samples]
X_test = X[-nb_validation_samples:]
Y_test = labels[-nb_validation_samples:]

Found 9578 unique tokens.


In [7]:
# подключаем словарь embedding-a , в котором каждому имеющемуся там слову сопоставлены 100 численных значений 
embeddings_index = {}  
f = open(os.path.join('/home/egor/projects/GoToSchoolAutumn', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()  # значения в словаре представлены словом и далее строкой чисел, поэтому сплитим их
    word = values[0]       # - ключ
    coefs = np.asarray(values[1:], dtype='float32') # 100 циферок
    embeddings_index[word] = coefs # тот же словарь , только со структурой ('слово':[зн-е1,зн-е2...зн-е100])
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM)) # делаем веса для embedding-слоя 

# итереруемся по всем уникальным словам из наших текстов и их индексам, которые сделал токенайзер
for word, i in word_index.items(): 
    embedding_vector = embeddings_index.get(word) # для слова берем те самые 100 значений из словаря
    
    if embedding_vector is not None:   
        # если слова в словаре нет - получаем вектор из 100 нулей
        embedding_matrix[i] = embedding_vector

# Модель


In [9]:
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM
from keras.layers.embeddings import Embedding
from sklearn.metrics import accuracy_score
from collections import Counter
from keras.regularizers import l1l2

Задаем веса классам т.е вычитаем из единицы долю класса во всех наблюдениях 

In [10]:
length = float(data.shape[0]) 
weights = Counter()
for cl in Y_l_encoded:
    weights[cl] += 1
    
values = np.array(weights.values())
keys = np.array(weights.keys())

new_values = 1 - values/length

class_weights = dict(zip(keys, new_values))

In [11]:
class_weights

{0: 0.96308129215477456,
 1: 0.92545260915867944,
 2: 0.94018459353922612,
 3: 0.87486687965921195,
 4: 0.94675186368477104,
 5: 0.91231806886758959,
 6: 0.94994675186368482,
 7: 0.96024139155129573,
 8: 0.91018814341498044,
 9: 0.85268015619453319,
 10: 0.9250976215832446,
 11: 0.8391906283280085}

Архитектура сетки

In [12]:
model = Sequential()

model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQ_LEN, weights=[embedding_matrix]))
model.add(LSTM(100))

model.add(Dense(64, activation='tanh', W_regularizer=l1l2(), b_regularizer=l1l2()))

model.add(Dense(12, activation='softmax'))


model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.save('model.h5')
del model

In [13]:
model = load_model('model.h5')


In [14]:
hist1 = model.fit(X_train,Y_train, nb_epoch=50,batch_size=200,class_weight=class_weights,validation_data=[X_test,Y_test])

Сохраняем веса, чтобы не обучать заново каждый раз после перезагрузки

In [15]:
model.save_weights('goodmodel1_weights.h5')

# Смотрим точность

Этой функцией "декодим" наш класс, т.е из вектора, состоящего из вероятностей принадлежности к каждому классу делаем вектор, определяющий класс

[0.066, 0.124, 0.071, 0.101, 0.027, 0.085, 0.064, 0.057, 0.068, 0.099, 0.053, 0.027, 0.128, 0.03] -> [0,0,0,0,0,0,0,0,0,0,0,0,1,0]   (13 класс)

In [16]:
def get_true_classes(pred):
    true_classes = []
    for row in pred:
        cl = np.zeros(12)
        index_of_max = row.argmax()

        cl[index_of_max] = 1

        true_classes.append(cl)
    true_classes = np.array(true_classes)
    return true_classes

Функция, которая считает accuracy для каждого класса

In [17]:
def my_accuracy(y_true,y_pred):
    cl_dict = {}
    length = y_true.shape[0]
      
    for i in range(length):
        if y_true[i].argmax() not in cl_dict:
        
            cl_dict[y_true[i].argmax()] = []
        
        if (y_true[i] == y_pred[i]).all():
            cl_dict[y_true[i].argmax()].append(1)
        else:
            cl_dict[y_true[i].argmax()].append(0)
     
    result = []
   
    for key,values in cl_dict.items():
        Sum = sum(values)
  
        accuracy = Sum/float(len(values))  
        result.append((key,accuracy))
        
    return result

Функция для красивого вывода accuracy 

In [18]:
def get_results(Y_true,Y_pred):
    results = my_accuracy(Y_true, Y_pred)

    return pd.DataFrame({'accuracy':map(lambda x: x[1],results)},
                             index=LE.inverse_transform(map(lambda x: x[0],results)))


Предсказываем вероятности (для train и для test) пренадлежности к классам и затем определяем эти классы для каждого вектора вероятностей. 

In [19]:
model.load_weights('goodmodel1_weights.h5')
train_pred = model.predict_proba(X_train)
true_pred_train = get_true_classes(train_pred)

test_pred = model.predict_proba(X_test)
true_pred_test = get_true_classes(test_pred)


1690/1690 [==============================] - 7s     


Точность (test) на каждом классе отдельно - отношение количества правильных предсказаний к количеству всех предсказаний для данного класса 

In [20]:
get_results(Y_test,true_pred_test)

,accuracy
Bash,0.391304
Blitzer,0.469231
Bush,0.597826
Clinton,0.748837
Cooper,0.714286
Cruz,0.728682
Kasich,0.637363
Kelly,0.647887
Rubio,0.705128
Sanders,0.871681


Выводим точность на train и на test.

In [21]:
pd.DataFrame({'accuracy':[accuracy_score(Y_train,true_pred_train),
              accuracy_score(Y_test,true_pred_test)]}, index=['train','test'])

,accuracy
train,0.710953
test,0.733136
